In [1]:
!pip install numpy
!pip install scipy
!pip install scikit-learn
!pip install joblib
!pip install numba
!pip install pyts
!pip install sktime
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 72.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyts.classification import SAXVSM
from pyts.classification import BOSSVS
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import f1_score
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.hybrid import HIVECOTEV2

In [3]:
X = np.load("drive/MyDrive/timeseries/train_x.npy")
y = np.load("drive/MyDrive/timeseries/train_y.npy")
X_submission = np.load("drive/MyDrive/timeseries/test_x.npy")

Classify using SAXVSM algorithm from pyts.

In [6]:
tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)
print(tscv)

max_metric = float('-inf')
best_estimator = None
metrics = []
for train_index, test_index in tscv.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  saxvsm = SAXVSM(window_size=32, word_size=8, n_bins=4, strategy='uniform')
  saxvsm.fit(X_train, y_train)
  y_pred = saxvsm.predict(X_test)
  metric = f1_score(y_test, y_pred, average='macro')
  if metric > max_metric:
    best_estimator = saxvsm
    max_metric = metric
  metrics.append(metric)
  print("F1 MACRO:", metric)
print(metrics)
y_submission = best_estimator.predict(X_submission)
answer = pd.DataFrame({
    'ID': np.array(range(y_submission.shape[0])),
    'Answer': y_submission.astype('int64'),
})
answer.to_csv("submission_SAXVSM.csv", index=False)
answer

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
TRAIN: [   0    1    2 ... 2444 2445 2446] TEST: [2447 2448 2449 ... 4888 4889 4890]
F1 MACRO: 0.2631812364615703
TRAIN: [   0    1    2 ... 4888 4889 4890] TEST: [4891 4892 4893 ... 7332 7333 7334]
F1 MACRO: 0.2587653456318252
TRAIN: [   0    1    2 ... 7332 7333 7334] TEST: [7335 7336 7337 ... 9776 9777 9778]
F1 MACRO: 0.21074573183011602
TRAIN: [   0    1    2 ... 9776 9777 9778] TEST: [ 9779  9780  9781 ... 12220 12221 12222]
F1 MACRO: 0.2406584260876988
TRAIN: [    0     1     2 ... 12220 12221 12222] TEST: [12223 12224 12225 ... 14664 14665 14666]
F1 MACRO: 0.20451071811923766
[0.2631812364615703, 0.2587653456318252, 0.21074573183011602, 0.2406584260876988, 0.20451071811923766]


,ID,Answer
0,0,3
1,1,3
2,2,3
3,3,2
4,4,2
...,...,...
7220,7220,3
7221,7221,3
7222,7222,3
7223,7223,3


Classify using ROCKET algorithm from sktime.

In [12]:
tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)
print(tscv)

max_metric = float('-inf')
best_estimator = None
metrics = []
for train_index, test_index in tscv.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  rocket = RocketClassifier(num_kernels=2000)
  rocket.fit(X_train, y_train)
  y_pred = rocket.predict(X_test)
  metric = f1_score(y_test, y_pred, average='macro')
  if metric > max_metric:
    best_estimator = rocket
    max_metric = metric
  metrics.append(metric)
  print("F1 MACRO:", metric)
print(metrics)
y_submission = best_estimator.predict(X_submission)
answer = pd.DataFrame({
    'ID': np.array(range(y_submission.shape[0])),
    'Answer': y_submission.astype('int64'),
})
answer.to_csv("submission_ROCKET.csv", index=False)
answer

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
TRAIN: [   0    1    2 ... 2444 2445 2446] TEST: [2447 2448 2449 ... 4888 4889 4890]
F1 MACRO: 0.8008224511133651
TRAIN: [   0    1    2 ... 4888 4889 4890] TEST: [4891 4892 4893 ... 7332 7333 7334]
F1 MACRO: 0.8296242562123595
TRAIN: [   0    1    2 ... 7332 7333 7334] TEST: [7335 7336 7337 ... 9776 9777 9778]
F1 MACRO: 0.9032370514521796
TRAIN: [   0    1    2 ... 9776 9777 9778] TEST: [ 9779  9780  9781 ... 12220 12221 12222]
F1 MACRO: 0.8719210999030432
TRAIN: [    0     1     2 ... 12220 12221 12222] TEST: [12223 12224 12225 ... 14664 14665 14666]
F1 MACRO: 0.886787789057521
[0.8008224511133651, 0.8296242562123595, 0.9032370514521796, 0.8719210999030432, 0.886787789057521]


,ID,Answer
0,0,0
1,1,0
2,2,0
3,3,2
4,4,0
...,...,...
7220,7220,0
7221,7221,2
7222,7222,0
7223,7223,0


Classify using HIVECOTE2 algorithm from sktime.

In [13]:
tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)
print(tscv)

max_metric = float('-inf')
best_estimator = None
metrics = []
for train_index, test_index in tscv.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  hc2 = HIVECOTEV2(time_limit_in_minutes=0.2)
  hc2.fit(X_train, y_train)
  y_pred = hc2.predict(X_test)
  metric = f1_score(y_test, y_pred, average='macro')
  if metric > max_metric:
    best_estimator = hc2
    max_metric = metric
  metrics.append(metric)
  print("F1 MACRO:", metric)
print(metrics)
y_submission = best_estimator.predict(X_submission)
answer = pd.DataFrame({
    'ID': np.array(range(y_submission.shape[0])),
    'Answer': y_submission.astype('int64'),
})
answer.to_csv("submission_HIVECOTE2.csv", index=False)
answer

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
TRAIN: [   0    1    2 ... 2444 2445 2446] TEST: [2447 2448 2449 ... 4888 4889 4890]
F1 MACRO: 0.762397072592015
TRAIN: [   0    1    2 ... 4888 4889 4890] TEST: [4891 4892 4893 ... 7332 7333 7334]
F1 MACRO: 0.7993944525886978
TRAIN: [   0    1    2 ... 7332 7333 7334] TEST: [7335 7336 7337 ... 9776 9777 9778]
F1 MACRO: 0.8454541266137431
TRAIN: [   0    1    2 ... 9776 9777 9778] TEST: [ 9779  9780  9781 ... 12220 12221 12222]
F1 MACRO: 0.8658492352285911
TRAIN: [    0     1     2 ... 12220 12221 12222] TEST: [12223 12224 12225 ... 14664 14665 14666]
F1 MACRO: 0.8627933936575646
[0.762397072592015, 0.7993944525886978, 0.8454541266137431, 0.8658492352285911, 0.8627933936575646]


,ID,Answer
0,0,0
1,1,0
2,2,0
3,3,2
4,4,0
...,...,...
7220,7220,0
7221,7221,2
7222,7222,0
7223,7223,0
